In [17]:
import numpy as np
import cupy as cp
import asyncio
import hashlib

class API:
    def __init__(self, size, int_range=(1, 10)):
        self.size = size
        self.matrices = []
        self.matrices.append(
            np.random.randint(int_range[0], int_range[1], (size, size))
        )
        self.matrices.append(
            np.random.randint(int_range[0], int_range[1], (size, size))
        )
        self.count = 0
        self.analytics: list[tuple[int, int, int]] = []

    def clear(self):
        self.count = 0

    async def call(self, endpoint):
        self.count += 1
        endpoint = endpoint.split("/")
        matrix = int(endpoint[1])
        type = endpoint[2]
        index = int(endpoint[3])
        
        await asyncio.sleep(0.01)

        return self.getRow(matrix, index) if type == "row" else self.getCol(matrix, index)

    def getRow(self, matrix, row) -> list[int]:
        return self.matrices[matrix][row, :]

    def getCol(self, matrix, col) -> list[int]:
        return self.matrices[matrix][:, col]
    
    def getHash(self):
        answer = np.dot(self.matrices[0], self.matrices[1])
        flattened = answer.flatten()
        joined = "".join([str(x) for x in flattened])
        return hashlib.md5(joined.encode()).hexdigest()
    
def calcHash(matrix):
    flattened = matrix.flatten()
    joined = "".join([str(x) for x in flattened])
    return hashlib.md5(joined.encode()).hexdigest()


In [18]:
api = API(1000)
api.getHash()

'62d23d9590376000adfd2bfdfe85764f'

In [19]:
async def fetch_compute(api, size):
    
    result = cp.zeros((size, size), dtype=int)

    row_tasks = [asyncio.ensure_future(api.call(f"/0/row/{i}")) for i in range(size)]
    col_tasks = [asyncio.ensure_future(api.call(f"/1/col/{i}")) for i in range(size)]

    rows: list[int] = await asyncio.gather(*row_tasks)
    cols: list[int] = await asyncio.gather(*col_tasks)

    # Converting rows and cols to CuPy arrays
    rows = cp.array(rows)
    # Transpose cols to make it easier to iterate
    cols = cp.array(cols).T

    result = cp.matmul(rows, cols)
    return result

async def compute_cell(a, b, col, row, result):
    # print(f"Computing cell {row}, {col}: {a} * {b} = {cp.matmul(a, b)}")
    result[row, col] = cp.matmul(a, b)

size = api.size
result = await fetch_compute(api, size)
# print(cp.asnumpy(result))
print(calcHash(cp.asnumpy(result)))
# print(np.dot(api.matrices[0], api.matrices[1]))
print(api.getHash())
print(api.count)

api.clear()

62d23d9590376000adfd2bfdfe85764f
62d23d9590376000adfd2bfdfe85764f
2000


In [23]:
# Using numpy to see if there is a difference

async def fetch_compute_np(api, size):
    result = cp.zeros((size, size), dtype=int)

    row_tasks = [asyncio.ensure_future(api.call(f"/0/row/{i}")) for i in range(size)]
    col_tasks = [asyncio.ensure_future(api.call(f"/1/col/{i}")) for i in range(size)]

    rows = await asyncio.gather(*row_tasks)  
    cols = await asyncio.gather(*col_tasks)
    
    rows = np.array(rows)
    cols = np.array(cols).T

    result = np.matmul(rows, cols)
    return result

np_result = await fetch_compute_np(api, size)
print(calcHash(np_result))
print(api.getHash())
print(api.count)

api.clear()

62d23d9590376000adfd2bfdfe85764f
62d23d9590376000adfd2bfdfe85764f
2000


In [20]:
api.matrices

validation = np.dot(api.matrices[0], api.matrices[1])
hash(validation.tobytes())

-8738621016486670724

In [21]:
print(api.count)

0


In [ ]:
test = np.random.rand(3, 3)
test

array([[0.85307433, 0.86669397, 0.65647558],
       [0.01481618, 0.5762393 , 0.35083939],
       [0.42792569, 0.34421946, 0.58111142]])

In [ ]:
z = np.array([[0.5, 0.2],[0.3, 0.7]])
x= np.array([[0.6, 0.4], [0.1, 0.9]])
np.dot(z, x)
# np.dot(z, x).tobytes()
# hash(np.dot(z, x).tobytes())

np.dot(z[0, :], x[:, 0])
np.dot(z[0, :], x[:, 1])
np.dot(z[1, :], x[:, 0])

0.25